In [3]:
import requests 

Running the sync job

In [13]:
url = "http://localhost:8000/api/v1/connections/sync"
data = {
    "connectionId": "3375812a-f8f0-4a6e-8bbd-6bb43ffdb73d"
}
response = requests.post(url=url, json=data)
print(response.status_code)
print(response.json())

200
{'job': {'id': 7, 'configType': 'sync', 'configId': '3375812a-f8f0-4a6e-8bbd-6bb43ffdb73d', 'createdAt': 1660970803, 'updatedAt': 1660970803, 'status': 'running'}, 'attempts': [{'attempt': {'id': 0, 'status': 'running', 'createdAt': 1660970803, 'updatedAt': 1660970803, 'streamStats': []}, 'logs': {'logLines': ['2022-08-20 04:46:43 \x1bINFO\x1b[m i.a.w.t.TemporalAttemptExecution(get):105 - Docker volume job log path: /tmp/workspace/7/0/logs.log', '2022-08-20 04:46:43 \x1bINFO\x1b[m i.a.w.t.TemporalAttemptExecution(get):110 - Executing worker wrapper. Airbyte version: 0.40.0-alpha']}}]}


In [14]:
# getting the job id 
job_id = response.json()["job"]["id"]
print(f"job_id: {job_id}")

job_id: 7


Getting the job status 

In [15]:
url = "http://localhost:8000/api/v1/jobs/get"
data = {
    "id": job_id
}
response = requests.post(url=url, json=data)

In [16]:
response.json()["job"]["status"]

'running'

Putting it all together 

In [4]:
import time 
import requests 
def sync_connection(connection_id:str)->bool:
    """
    Triggers an airbyte sync. 

    Input: 
    - connection_id: the id of the connection 

    Returns: 
    - bool: true if the sync succeeds, an error otherwise. 
    """
    url = "http://localhost:8000/api/v1/connections/sync"
    data = {
        "connectionId": connection_id
    }
    sync_response = requests.post(url=url, json=data)
    job_id = sync_response.json()["job"]["id"]
    job_status = "running"
    while job_status == "running": 
        time.sleep(5)
        url = "http://localhost:8000/api/v1/jobs/get"
        data = {
            "id": job_id
        }
        job_response = requests.post(url=url, json=data)
        job_status = job_response.json()["job"]["status"]
        if job_status == "failure": 
            raise Exception(f"Run failed. {job_response.text}")
    return True 

In [5]:
sync_connection(connection_id="3375812a-f8f0-4a6e-8bbd-6bb43ffdb73d")

True